<h1 id="tocheading">Оглавление</h1>
<div id="toc"></div>

In [3]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')


<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import pandas as pd
import pymysql
import os, codecs, ast
import plotly
import plotly.graph_objs as go
from plotly.offline import *
from plotly import tools
import requests

# Принудительная синхронизация баз

In [79]:
requests.get('http://parser.ommet.com/get_data.php')

<Response [200]>

# Параметры подключения к базе данных

In [2]:
def get_sql_data(sql_string):
    host   = '10.10.0.55'#'10.10.0.12'
    port   = 3306
    user   = 'parser_new'#'pyton'
    passwd = 'Cp5VLWmYr5xRKUQv'#'URLNDQhMGy3H3dDQ'
    db     = 'parser_new'#'parser_data'
    
    cnx = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db)
    
    df = pd.read_sql(sql_string, cnx)
    
    cnx.close()
    
    return df

In [3]:
def kolvo(x):
    return x.count()


# Проект Оммет-металл


In [4]:
def get_data_Omsk(days, cbp, ss):
    s="""SELECT i.id_system id, c.caption cat, i.caption tovar, s.caption mag, r.price, r.date_key date
    FROM items i
    INNER JOIN categories c ON i.id_category = c.id
    INNER JOIN items_results r ON r.id_items = i.id
    INNER JOIN sources s ON s.id = r.id_sources
    WHERE r.id_projects = 0x81afe3461c5e45ce828d3a2e32ab03d5 and s.caption not like '%-НСК%' and r.price>0""" 
    if cbp==0:
        s+=""" and s.caption <> 'Оммет-ЦБП'"""
    if ss==0:
        s+=""" and s.caption <> 'СПК-СС'"""
    if days==0:
        s+=' AND r.date_key = (select max(date_key) from items_results where id_projects = 0x81afe3461c5e45ce828d3a2e32ab03d5)'
    else:
        s+=' AND to_days(now())-to_days(r.date_key)<='+str(days)
    s+=' order by r.date_key'
    
    df=get_sql_data(s)
    
    #удалить товары, которые есть только у нас
    df_kol=df.copy()
    df_kol.drop(['cat','tovar','price','date'], axis=1)
    df_kol=df_kol[(df_kol.mag!='Оммет') & (df_kol.mag!='Оммет-ЦБП')]
    df_kol=df_kol.groupby(['id']).mag.agg([kolvo]).reset_index()
    df=pd.merge(df, df_kol, on=['id'],how='inner',suffixes=('_1','_2'))
    df=df.drop(['kolvo'], axis=1)
    
    return df

### Цены конкурентов ниже наших ЦБП

In [94]:
data = get_data_Omsk(days=0, cbp=1, ss=0)
data=data.dropna(subset=['id']).drop(['date'],axis=1)
data2=data.copy()
#выделяем в отдельный датасет данные по ЦБП
data2=data2[data2.mag=='Оммет-ЦБП']
data2=data2.drop(['cat','tovar','mag'],axis=1)
data=data[data.mag!='Оммет-ЦБП']
#удаляем данные по ЦБП из основного датасета и приклеиваем второй датасет отдельным столбцом
res=pd.merge(data, data2, on='id', how='inner',suffixes=('_1','_2'))
res=res[res.price_1<res.price_2].sort_values(by=['mag', 'cat', 'tovar']).reset_index()
res=res.drop(['index'],axis=1).rename(index=str, columns={"price_1": "Цена", "price_2": "ЦБП","mag":"Магазин"})
res


,id,cat,tovar,Магазин,Цена,ЦБП
0,3R7А1,Лист х/к оцинкованный,"Лист оцинкованный 1,50х1250х2500",Леман55,56500.00,58155.00
1,3UYА1,Полоса стальная горячекатаная,Полоса горячекатаная 4х40 Ст3сп/пс,Леруа,46974.52,49410.00
2,45GА1,Труба квадратная,Труба электросварная квадратная 60х60х3,Леруа,30984.13,47100.95
3,485А1,Труба прямоугольная,"Труба электросварная прямоугол 40х20х1,5",Леруа,51653.94,55099.24
4,314А1,Арматура а3 А500 прутки,Арматура рифленая 12 А500 СП,Мечел-ОМС,35300.00,40000.00
5,3VSА1,Проволока ВР-1,Проволока Вр-1 3мм,Мечел-ОМС,44500.00,45580.00
6,44XА1,Труба квадратная,"Труба электросварная квадратная 20х20х1,5",Мечел-ОМС,55100.00,55166.67
7,451А1,Труба квадратная,"Труба электросварная квадратная 25х25х1,5",Мечел-ОМС,55100.00,55102.80
8,456А1,Труба квадратная,"Труба электросварная квадратная 40х40х1,5",Мечел-ОМС,55100.00,55101.12
9,484А1,Труба прямоугольная,"Труба электросварная прямоугол 30х20х1,5",Мечел-ОМС,55100.00,55101.85


### Графики цен конкурентов по Омску

In [13]:
data = get_data_Omsk(days=7, cbp=1, ss=1)
data=data.dropna(subset=['cat'])
data_cat=data.copy().drop(['id','tovar','mag','price','date'],axis=1)
data_cat=data_cat.groupby(['cat']).head(1).reset_index()
i=0
data_cat['prod']=''
for index in data_cat['cat'].values:   
    data_cat['prod'].loc[i]=data_cat['cat'].loc[i]
    i+=1
data=pd.merge(data, data_cat, on='cat', how='inner',suffixes=('_1','_2'))
data=data.drop(['id','tovar','cat'],axis=1).groupby(['prod','mag','date']).mean() #grouped dataframe
data=data.reset_index() #ungrouped dataframe

data2=data.copy()
dat=[]
dat_list=[]
ttl=[]
prod=''
while not data2.empty:
    #как только категория поменялась - выпихнуть график и сбросить список серий
    #график рисуется если есть хоть кто-то кроме оммета
    if prod!=data2['prod'].iloc[0]:
        if len(dat)>1:
            dat_list.append(dat)
            if len(prod)>30:
                a=prod[:30].rfind(' ')
                prod=prod[:a-1]+'\n'+prod[a+1:]
            ttl.append(prod)
        dat=[]
    prod=data2['prod'].iloc[0]
    mag=data2['mag'].iloc[0]
    #взять копию оставшегося датасета и отобрать из нее текущие категорию и контрагента
    #из оставшегося датасета эти данные удалить
    data3=data2.copy()
    data3=data3[(data3['mag']==mag) & (data3['prod']==prod)]
    data2=data2[(data2['mag']!=mag) | (data2['prod']!=prod)]
    #из полученных данных создать серию для графика и загнать в список
    trace = go.Scatter(
        x=data3['date'],
        y=data3['price'],
        mode='lines',
        name=mag
    )
    dat.append(trace)
#выпихнуть последний график
if len(dat)>1:
    dat_list.append(dat)
    ttl.append(prod)

fig = tools.make_subplots(rows=int(len(dat_list)/3+1), cols=3, subplot_titles=ttl)
i=0
for tr in dat_list:
    for obj in tr:
        fig.append_trace(obj, int(i/3+1), int(i%3+1))
    i+=1

#layout = go.Layout(showlegend=False)
fig['layout'].update(height=int(len(dat_list)/3+1)*400, title='Графики цен конкурентов по Омску', showlegend=False)
plotly.offline.plot(fig, filename='omsk_price.html')

C:\Users\itc\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



This is the format of your plot grid:
[ (1,1) x1,y1 ]     [ (1,2) x2,y2 ]     [ (1,3) x3,y3 ]   
[ (2,1) x4,y4 ]     [ (2,2) x5,y5 ]     [ (2,3) x6,y6 ]   
[ (3,1) x7,y7 ]     [ (3,2) x8,y8 ]     [ (3,3) x9,y9 ]   
[ (4,1) x10,y10 ]   [ (4,2) x11,y11 ]   [ (4,3) x12,y12 ] 
[ (5,1) x13,y13 ]   [ (5,2) x14,y14 ]   [ (5,3) x15,y15 ] 
[ (6,1) x16,y16 ]   [ (6,2) x17,y17 ]   [ (6,3) x18,y18 ] 
[ (7,1) x19,y19 ]   [ (7,2) x20,y20 ]   [ (7,3) x21,y21 ] 
[ (8,1) x22,y22 ]   [ (8,2) x23,y23 ]   [ (8,3) x24,y24 ] 
[ (9,1) x25,y25 ]   [ (9,2) x26,y26 ]   [ (9,3) x27,y27 ] 
[ (10,1) x28,y28 ]  [ (10,2) x29,y29 ]  [ (10,3) x30,y30 ]
[ (11,1) x31,y31 ]  [ (11,2) x32,y32 ]  [ (11,3) x33,y33 ]
[ (12,1) x34,y34 ]  [ (12,2) x35,y35 ]  [ (12,3) x36,y36 ]
[ (13,1) x37,y37 ]  [ (13,2) x38,y38 ]  [ (13,3) x39,y39 ]
[ (14,1) x40,y40 ]  [ (14,2) x41,y41 ]  [ (14,3) x42,y42 ]
[ (15,1) x43,y43 ]  [ (15,2) x44,y44 ]  [ (15,3) x45,y45 ]
[ (16,1) x46,y46 ]  [ (16,2) x47,y47 ]  [ (16,3) x48,y48 ]
[ (17,1) x49,y49 ]

'omsk_price.html'

### Подозрения конкурентов в сговоре

In [96]:
data = get_data_Omsk(days=7, cbp=0, ss=0)
data=data.dropna(subset=['id'])
#оставляем всех омских контрагентов кроме оммета
data=data[data.mag!='Оммет'].drop(['date'],axis=1)
data2=data.copy()
data2=data2.drop(['cat','tovar'],axis=1)
#создаем попарный список контрагентов для совпадающих товаров
res=pd.merge(data, data2, on='id', how='inner',suffixes=('_1','_2'))
res=res[(res.mag_1<res.mag_2) & (res.price_1!=0) & (res.price_2!=0)]
#усредняем цены в списке по категориям
res=res.drop(['tovar'],axis=1).groupby(['cat','mag_1','mag_2']).mean()
#в итоге если разница цен по отношению к средней меньше...
res[abs((res.price_1-res.price_2)/(res.price_1+res.price_2))<0.001]


price_1  \
cat                            mag_1      mag_2                          
Квадрат г/к Ст3                Леман55    Промметалл      54250.000000   
Лист х/к из качественной стали Леман55    Промметалл      54966.666667   
Полоса стальная горячекатаная  Леман55    Пухов           54000.000000   
                               Промметалл СПК-ОМС (10тн)  54325.000000   
Труба квадратная               Мечел-ОМС  Промметалл      51350.000000   
Труба прямоугольная            Леруа      ОСИ             65521.625000   

                                                               price_2  
cat                            mag_1      mag_2                         
Квадрат г/к Ст3                Леман55    Промметалл      54350.000000  
Лист х/к из качественной стали Леман55    Промметалл      54900.000000  
Полоса стальная горячекатаная  Леман55    Пухов           54000.000000  
                               Промметалл СПК-ОМС (10тн)  54324.250000  
Труба квадратная               Мечел-ОМС  Промметалл      51260.714286  
Труба прямоугольная            Леруа      ОСИ             65648.850000

### Конкурентность товара по Омску


In [58]:
def mag_kolvo(x):
    return x.size

def min_price(x):
    return x.min()

def max_price(x):
    return x.max()


In [59]:
data = get_data_Omsk(days=7, cbp=0, ss=0)
data=data.dropna(subset=['id'])
#усредняем цены по датам
data=data.drop(['date'],axis=1)
data=data.groupby(['id','cat','tovar','mag']).mean()
data=data.reset_index()
#выделяем оммет в отдельный датасет
data_ommet=data.copy()
data_ommet=data_ommet[data_ommet.mag==u'Оммет'].drop(['mag'],axis=1)
data=data[data.mag!=u'Оммет'].drop(['mag'],axis=1)
#расчет кол-ва конкурентов и диапазона цен
data=data.groupby(['id','cat','tovar']).price.agg([mag_kolvo, min_price, max_price])
data=data.reset_index().drop(['cat','tovar'],axis=1)
#сравниваем с нашими ценами
res=pd.merge(data_ommet, data, on='id', how='outer',suffixes=('_1','_2'))
res=res.rename(index=str, columns={"price": "ommet_price"})
res


,id,cat,tovar,ommet_price,mag_kolvo,min_price,max_price
0,30VА1,Арматура рифленая прутковая обыкновенного каче...,Арматура рифленая 10 А500 СП,44000.000000,10.0,42971.000000,60958.038333
1,30ZА1,Арматура рифленая прутковая повышенной прочности,Арматура рифленая 10 а3 35ГС,44700.000000,2.0,43750.990000,45501.030000
2,314А1,Арматура рифленая прутковая обыкновенного каче...,Арматура рифленая 12 А500 СП,40400.000000,10.0,35266.666667,61561.776667
3,316А1,Арматура рифленая прутковая повышенной прочности,Арматура рифленая 12 а3 25Г2С,41400.000000,3.0,40450.940000,42533.333333
4,31DА1,Арматура рифленая прутковая обыкновенного каче...,Арматура рифленая 14 А500 СП,40000.000000,7.0,39071.040000,60101.280000
5,31FА1,Арматура рифленая прутковая повышенной прочности,Арматура рифленая 14 а3 25Г2С,41000.000000,3.0,40090.931667,42033.333333
6,31NА1,Арматура рифленая прутковая обыкновенного каче...,Арматура рифленая 16 А500 СП,40000.000000,7.0,39071.040000,58719.983333
7,31PА1,Арматура рифленая прутковая повышенной прочности,Арматура рифленая 16 а3 25Г2С,41000.000000,3.0,40090.931667,42033.333333
8,31VА1,Арматура рифленая прутковая обыкновенного каче...,Арматура рифленая 18 А500 СП,40000.000000,4.0,39071.040000,42800.000000
9,322А1,Арматура рифленая прутковая обыкновенного каче...,Арматура рифленая 20 А500 СП,40000.000000,4.0,39071.040000,44502.305000


# Проект MetalSee

In [5]:
def get_data_MetalSee(days, cbp):
    s="""SELECT i.id_system id, c.caption cat, i.caption tovar, s.caption mag, r.price, r.date_key date, co.okrug_name okrug
    FROM items i
    INNER JOIN categories c ON i.id_category = c.id
    INNER JOIN items_results r ON r.id_items = i.id
    INNER JOIN sources s ON s.id = r.id_sources
    LEFT JOIN city_okrug co on s.caption like concat('%',co.city_short)
    WHERE r.id_projects = 0x12105106d0d4440e9e4d0fe93ae98442 and r.price>0 """
    if cbp==0:
        s+=""" and s.caption <> 'ЦБП-ОМС'"""
    if days==0:
        s+=' and r.date_key = (select max(date_key) from items_results where id_projects = 0x12105106d0d4440e9e4d0fe93ae98442)'
    else:
        s+=' and to_days(now())-to_days(r.date_key)<='+str(days)
    s+=' order by r.date_key'
    
    return get_sql_data(s)

### Графики динамики цен конкурентов (среднее по продукции в городе)

In [8]:
data = get_data_MetalSee(days=7, cbp=0)

#вычислить город из задней части контрагента
data['reg']=data['mag'].apply(lambda x: x[x.find('-')+1 :])
data=data.drop(['id','tovar','mag','okrug'],axis=1).groupby(['cat','reg','date']).mean() #grouped dataframe
data=data.reset_index() #ungrouped dataframe

data2=data.copy()
dat=[]
dat_list=[]
ttl=[]
cat=''
while not data2.empty:
    #как только категория поменялась - выпихнуть график и сбросить список серий
    if cat!=data2.cat.iloc[0]:
        if len(dat)>0:
            dat_list.append(dat)
            ttl.append(cat)
            dat=[]
    cat=data2.cat.iloc[0]
    reg=data2.reg.iloc[0]
    #взять копию оставшегося датасета и отобрать из нее текущие категорию и контрагента
    #из оставшегося датасета эти данные удалить
    data3=data2.copy()
    data3=data3[(data3.reg==reg) & (data3.cat==cat)]
    data2=data2[(data2.reg!=reg) | (data2.cat!=cat)]
    #из полученных данных создать серию для графика и загнать в список
    trace = go.Scatter(
        x=data3['date'],
        y=data3['price'],
        mode='lines',
        name=reg
    )
    dat.append(trace)
#выпихнуть последний график
dat_list.append(dat)
ttl.append(cat)

fig = tools.make_subplots(rows=int(len(dat_list)/3+1), cols=3,subplot_titles=ttl)
i=0
for tr in dat_list:
    for obj in tr:
        fig.append_trace(obj, int(i/3+1), int(i%3+1))
    i+=1

fig['layout'].update(height=int(len(dat_list)/3+1)*400, title='Графики динамики цен конкурентов (среднее по продукции в городе)', showlegend=False)
plotly.offline.plot(fig, filename='region_price.html')

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]    [ (1,3) x3,y3 ]  
[ (2,1) x4,y4 ]    [ (2,2) x5,y5 ]    [ (2,3) x6,y6 ]  
[ (3,1) x7,y7 ]    [ (3,2) x8,y8 ]    [ (3,3) x9,y9 ]  
[ (4,1) x10,y10 ]  [ (4,2) x11,y11 ]  [ (4,3) x12,y12 ]



'region_price.html'

### Графики динамики цен конкурентов (среднее по продукции  в округе)

In [9]:
data = get_data_MetalSee(days=7, cbp=0)

data=data.drop(['id','tovar','mag'],axis=1).groupby(['cat','date','okrug']).mean() #grouped dataframe
data=data.reset_index() #ungrouped dataframe

data2=data.copy()
dat=[]
dat_list=[]
ttl=[]
cat=''
while not data2.empty:
    #как только категория поменялась - выпихнуть график и сбросить список серий
    if cat!=data2.cat.iloc[0]:
        if len(dat)>0:
            dat_list.append(dat)
            ttl.append(cat)
            dat=[]
    cat=data2.cat.iloc[0]
    okrug=data2.okrug.iloc[0]
    #взять копию оставшегося датасета и отобрать из нее текущие категорию и региона
    #из оставшегося датасета эти данные удалить
    data3=data2.copy()
    data3=data3[(data3.okrug==okrug) & (data3.cat==cat)]
    data2=data2[(data2.okrug!=okrug) | (data2.cat!=cat)]
    #из полученных данных создать серию для графика и загнать в список
    trace = go.Scatter(
        x=data3['date'],
        y=data3['price'],
        mode='lines',
        name=okrug
    )
    dat.append(trace)
#выпихнуть последний график
dat_list.append(dat)
ttl.append(cat)

fig = tools.make_subplots(rows=int(len(dat_list)/3+1), cols=3,subplot_titles=ttl)
i=0
for tr in dat_list:
    for obj in tr:
        fig.append_trace(obj, int(i/3+1), int(i%3+1))
    i+=1

fig['layout'].update(height=int(len(dat_list)/3+1)*400, title='Графики динамики цен конкурентов (среднее по продукции в округе)', showlegend=False)
plotly.offline.plot(fig, filename='okrug_price.html')

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]    [ (1,3) x3,y3 ]  
[ (2,1) x4,y4 ]    [ (2,2) x5,y5 ]    [ (2,3) x6,y6 ]  
[ (3,1) x7,y7 ]    [ (3,2) x8,y8 ]    [ (3,3) x9,y9 ]  
[ (4,1) x10,y10 ]  [ (4,2) x11,y11 ]  [ (4,3) x12,y12 ]



'okrug_price.html'

### Изменения ценника больше определенного процента по всем товарам и всем конкурентам.

In [12]:
from datetime import datetime, timedelta

data = get_data_MetalSee(days=7, cbp=0)
data=data.drop(['okrug'],axis=1).dropna(subset=['id'])
data2=data.copy()
data2=data2.drop(['cat','tovar'],axis=1)
#создаем попарный список цен за соседние дни
res=pd.merge(data, data2, on=['id','mag'], how='inner',suffixes=('_1','_2'))
res=res[res.date_2.subtract(res.date_1)==timedelta(days=1)]
res['otklon']=100*(1-res.price_1/res.price_2)
res=res[abs(res.otklon)>5].reset_index()
res


,index,id,cat,tovar,mag,price_1,date_1,price_2,date_2,otklon
0,7386,30VА1,Арматура,Арматура рифленая 10 А500 СП,А групп-РОВ,41000.00,2019-07-02,44000.0,2019-07-03,6.818182
1,9934,43RА1,Труба,"Труба ВГП 20х2,8",А групп-КАЯ,50000.00,2019-07-03,55000.0,2019-07-04,9.090909
2,11032,43RА1,Труба,"Труба ВГП 20х2,8",МС-МСК,44490.00,2019-07-05,48490.0,2019-07-06,8.249124
3,13025,43RА1,Труба,"Труба ВГП 20х2,8",МС-ЧЛБ,46400.00,2019-07-04,48990.0,2019-07-05,5.286793
4,13589,3I7А1,Круг,Круг горячекатаный 20 Ст3,Мечел-ЕКБ,44400.00,2019-07-01,46900.0,2019-07-02,5.330490
5,13867,43RА1,Труба,"Труба ВГП 20х2,8",МС-ЕКБ,46400.00,2019-07-04,48990.0,2019-07-05,5.286793
6,17730,43ZА1,Труба,"Труба ВГП 50х3,5",А групп-КАЯ,48200.00,2019-07-03,53000.0,2019-07-04,9.056604
7,17738,43ZА1,Труба,"Труба ВГП 50х3,5",А групп-КАЯ,53000.00,2019-07-04,48200.0,2019-07-05,-9.958506
8,17771,4AMА1,Угол,Уголок горячекатаный 35х35х4 Ст3,СПК-КАЯ,50664.50,2019-07-02,54356.0,2019-07-03,6.791339
9,18473,43ZА1,Труба,"Труба ВГП 50х3,5",А групп-МСК,46000.00,2019-07-04,42000.0,2019-07-05,-9.523810


### Графики динамики цен конкурентов (Аналитика по округам и городам)

In [6]:
def price_mean(x):
    return x.mean()

def price_median(x):
    return x.median()


In [21]:
from scipy import stats
from sklearn.preprocessing import scale
from sklearn import svm
from sklearn.decomposition import PCA

data = get_data_MetalSee(days=30, cbp=1)
data_kol=data.copy()
data_kol.drop(['cat','tovar','okrug','price'], axis=1)
data_kol=data_kol.groupby(['id','date']).mag.agg([kolvo])
data_kol=data_kol[data_kol.kolvo>1].reset_index()
data=pd.merge(data, data_kol, on=['id','date'],how='inner',suffixes=('_1','_2'))

#поиск выбросов
#i=0
#df_qr=data.groupby(['cat','mag']).price.agg([kolvo]).reset_index()
#df_qr['cat_mag']=df_qr['cat']+df_qr['mag']
#for index in df_qr['cat_mag'].values:
#    df=data[(data.cat==df_qr['cat'].loc[i]) & (data.mag==df_qr['mag'].loc[i])]
#    q1=df['price'].quantile(0.25)
#    q3=df['price'].quantile(0.75)
#    iqr = q3-q1 #Interquartile range
#    fence_low  = q1-1.5*iqr
#    fence_high = q3+1.5*iqr
#    df = df[(df['price'] < fence_low) | (df['price'] > fence_high)]
#    df=df[['id','mag','date','price']]
#    data=pd.merge(data, df, on=['id','mag','date'],how='outer',suffixes=('','_2'))
#    data[['price_2']] = data[['price_2']].fillna(value=0)
#    data=data[data.price_2==0].reset_index().drop(['price_2','index'],axis=1)
#    i+=1

data=data.drop(['id','tovar','kolvo'], axis=1)
data=data.sort_values(by=['cat']).reset_index()

mean_list=[]
okrug_list=[]
okrug_full_list=[]
city_list=[]
city_full_list=[]
ttl=[]
nGrLines=0

cat=''
while not data.empty:
    cat=data.cat.iloc[0]
    full_data=data[data.cat==cat].copy()
    data=data[data.cat!=cat]
    full_data=full_data.drop(['cat'],axis=1)
    
    #поиск выбросов

    prm = np.array(full_data.price.values, dtype="float64")
    prm=prm.reshape((prm.size,1))
    prm = scale(prm)
    X = PCA(n_components=1).fit_transform(prm)
    clf = svm.OneClassSVM(kernel="rbf")
    clf.fit(X)
    dist_to_border = clf.decision_function(X).ravel()
    threshold = stats.scoreatpercentile(dist_to_border,1)
    is_inlier = (dist_to_border > threshold)
    l=[]
    for i in range(len(is_inlier)):
        if is_inlier[i]==False:
            l.append(i)
    print(l)
    full_data = full_data[~full_data.index.isin(l)]

    #full_data=full_data[((full_data.price - full_data.price.mean())/full_data.price.std()).abs() < 3] #удалить выбросы по 3-сигмам
    
    mean_data=full_data.copy()
    mean_data=mean_data[mean_data.mag!='ЦБП-ОМС']
    mean_data=mean_data.drop(['mag','okrug'],axis=1)
    mean_data=mean_data.groupby(['date']).price.agg([price_mean, price_median]).reset_index()
    l=[]
    trace = go.Scatter(
        x=mean_data['date'],
        y=mean_data['price_mean'],
        mode='lines',
        name='Мат.ожидание'
    )
    l.append(trace)
    
    trace = go.Scatter(
        x=mean_data['date'],
        y=mean_data['price_median'],
        mode='lines',
        name='Медиана'
    )
    l.append(trace)
    
    mean_list.append(l)
    ttl.append('')
    ttl.append(cat)
    ttl.append('')
    nGrLines+=1
    l=[]
    #------------------------------------------------------------------------------------
    okrug_data=full_data.copy()
    okrug_data=okrug_data[okrug_data.mag!='ЦБП-ОМС']
    #вычислить город из задней части контрагента
    okrug_data['reg']=okrug_data['mag'].apply(lambda x: x[x.find('-')+1 :])
    okrug_data=okrug_data.drop(['mag'],axis=1)
    okrug_data=okrug_data.groupby(['okrug','reg','date']).mean().reset_index()
    okrug=''
    okrug_list=[]
    while not okrug_data.empty:
        if okrug!=okrug_data.okrug.iloc[0]:
            if len(l)>0:
                okrug_list.append(l)
                ttl.append(okrug)
                l=[]
            okrug=okrug_data.okrug.iloc[0]
        reg=okrug_data.reg.iloc[0]
        dt_copy=okrug_data.copy()
        dt_copy=dt_copy[(dt_copy.reg==reg) & (dt_copy.okrug==okrug)]
        okrug_data=okrug_data[(okrug_data.reg!=reg) | (okrug_data.okrug!=okrug)]
        #из полученных данных создать серию для графика и загнать в список
        trace = go.Scatter(
            x=dt_copy['date'],
            y=dt_copy['price'],
            mode='lines',
            name=reg
        )
        l.append(trace)
    #выпихнуть последний график
    okrug_list.append(l)
    ttl.append(okrug)
    okrug_full_list.append(okrug_list)
    n=len(okrug_list)
    while (n%3)>0:
        ttl.append('')
        n+=1
    nGrLines+=int(n/3)
    #------------------------------------------------------------------------------------
    l=[]
    city_data=full_data.copy()
    #вычислить город из задней части контрагента
    city_data['reg']=city_data['mag'].apply(lambda x: x[x.find('-')+1 :])
    city_data=city_data.drop(['okrug'],axis=1)
    city_data=city_data.groupby(['reg','mag','date']).mean().reset_index()
    reg=''
    city_list=[]
    while not city_data.empty:
        if reg!=city_data.reg.iloc[0]:
            if len(l)>0:
                city_list.append(l)
                ttl.append(reg)
                l=[]
            reg=city_data.reg.iloc[0]
        mag=city_data.mag.iloc[0]
        dt_copy=city_data.copy()
        dt_copy=dt_copy[(dt_copy.reg==reg) & (dt_copy.mag==mag)]
        city_data=city_data[(city_data.reg!=reg) | (city_data.mag!=mag)]
        #из полученных данных создать серию для графика и загнать в список
        trace = go.Scatter(
            x=dt_copy['date'],
            y=dt_copy['price'],
            mode='lines',
            name=mag
        )
        l.append(trace)
    #выпихнуть последний график
    city_list.append(l)
    ttl.append(reg)
    city_full_list.append(city_list)
    n=len(city_list)
    while (n%3)>0:
        ttl.append('')
        n+=1
    nGrLines+=int(n/3)
    
fig = tools.make_subplots(rows=nGrLines, cols=3,subplot_titles=ttl)
i=0
n=0
for tr_m in mean_list:
    for obj_m in tr_m:
        fig.append_trace(obj_m, int(i/3+1), 2)
    i+=3
    for tr_o in okrug_full_list[n]:
        for obj_o in tr_o:
            fig.append_trace(obj_o, int(i/3+1), int(i%3+1))
        i+=1
    i+=((3-i%3)%3)
    for tr_c in city_full_list[n]:
        for obj_c in tr_c:
            fig.append_trace(obj_c, int(i/3+1), int(i%3+1))
        i+=1
    i+=((3-i%3)%3)
    n+=1

fig['layout'].update(height=400*i/3, title='Аналитика', showlegend=False)
plotly.offline.plot(fig, filename='graphics.html')

[79, 80, 85, 161, 188, 190, 223, 275, 276, 278, 511, 540, 568, 952, 1126, 1131, 1192, 1194, 1195, 1197, 1198, 1200, 1202, 1561, 1811, 1905, 2023, 2027, 2034, 2071, 2164, 2257, 2676, 2677, 2682, 2869, 2871, 2872, 3278, 3296, 3299, 3303, 3598, 3642, 3658, 3659, 3685, 3690, 3824, 3826, 3849, 3850, 3851, 3878, 4383, 4414, 4483, 4582, 4635, 4638, 4652, 5152, 5156, 5160, 5163, 5469, 5487, 5488, 5490, 5524, 5528, 5567, 5575, 5577, 5650, 5656]
[132, 191, 255, 375, 377, 378, 384, 393, 812, 1009, 1011, 1017, 1018, 1020, 1146, 1180, 1679, 1681, 1732, 1734, 1735, 1897, 2106, 2130, 2132, 2341, 2343, 2465, 2466, 2559, 2686, 2702, 2872, 2875, 2883, 2884, 2900, 3139, 3140, 3146, 3149, 3155, 3594, 3622, 3624, 3864, 3884, 3885, 3909, 3911, 4238, 4436, 4438, 4439, 4529, 4531, 4584, 4610, 4612, 4839, 4873, 4932, 4934, 4935, 5064, 5066, 5072, 5073, 5075, 5135, 5137, 5144, 5146, 5152, 5280, 5328, 5517, 6180, 6185, 6189, 6402, 6404, 6405, 6412, 6450, 6512, 6514, 6520, 6521, 6523, 6704, 6705, 6714, 6810, 6812

'graphics.html'